In [1]:
import numpy as np
import pandas as pd
import os as os

In [2]:
os.chdir('/Users/raghuramkowdeed/Documents/afp/')

In [3]:
%pylab inline
exec(open("./run_model.py").read())
exec(open("./get_signal_names.py").read())
exec(open("./portfolio_optimizer.py").read())

import datetime
from pylab import rcParams


Populating the interactive namespace from numpy and matplotlib


In [4]:
use_rank = True

sig_dir = '/Users/raghuramkowdeed/Desktop/afp_data/rank/500/rps_data/'
fac_data_dir='/Users/raghuramkowdeed/Desktop/afp_data/rank/500/factor_exposure_data/'
fac_ret_file = '/Users/raghuramkowdeed/Desktop/afp_data/rank/500/fac_ret_file.csv'

if not use_rank:
    sig_dir = '/Users/raghuramkowdeed/Desktop/afp_data/norm/500/rps_data/'
    fac_data_dir='/Users/raghuramkowdeed/Desktop/afp_data/norm/500/factor_exposure_data/'
    fac_ret_file = '/Users/raghuramkowdeed/Desktop/afp_data/norm/500/fac_ret_file.csv'

In [5]:
sig_df = pd.DataFrame()
sig_files = os.listdir(sig_dir)
sig_files = np.sort(sig_files)

for sig_file in sig_files :
    if '.csv' not in sig_file :
        continue
    #print(sig_file)
    this_file = sig_dir + sig_file
    this_df = pd.read_csv(this_file, index_col = 'date')
    this_df.index = [ datetime.datetime.strptime(val, '%Y-%m-%d') for val in this_df.index]
    sig_df = sig_df.append(this_df)
data = sig_df.copy()

In [6]:
model_name = 'pls_regression'
train_model_arg = { 'n_components':50, 'scale':False, 'max_iter':5000, 'tol':1e-08, 'copy':True }
#model_name = 'linear_regression'
#train_model_arg = None


look_back = 12

In [7]:
begin_date = datetime.datetime.strptime('19900101', '%Y%m%d')
end_date = datetime.datetime.strptime('20170101', '%Y%m%d')

sig_df = sig_df[(sig_df.index>=begin_date) & (sig_df.index<end_date)]
sig_df = sig_df.fillna(sig_df.mean())

In [8]:
'res_ret' in sig_df.columns

True

In [9]:
#factor_names =  ['beta','mve', 'bm', 'roic', 'agr', 'mom12m', 'idiovol']
factor_names =  ['beta']
sig_names = get_signal_names()
ret_sr = sig_df['ret']
sig_df = sig_df[sig_names]
sig_df = sig_df.drop(factor_names, axis=1)

In [ ]:
#dim_vec = range(5,90,10)
dim_vec = range(1,25,1)

In [ ]:
model_sig_df = pd.DataFrame()

for this_dim in dim_vec :
    print(this_dim)
    train_model_arg['n_components'] = this_dim    
    model_sig_sr = run_model( sig_df, ret_sr, model_name, train_model_arg=train_model_arg, 
                          pred_model_arg ={},look_back=look_back)
    model_sig_df[this_dim] = model_sig_sr

1
2
3
4


In [ ]:
'res_ret' in sig_df.columns

In [ ]:

pm = PortfolioOptimizer(fac_data_dir=fac_data_dir,
                        fac_ret_file = fac_ret_file,
                        fac_cov_hl = 12.0,
                        fac_names = None
                       )



In [ ]:
#new_sig_df = pd.DataFrame()

#for sig_name in sig_df.columns :
#    this_sig = sig_df[sig_name]
#    this_sig = pm.neu_signal(this_sig)
#    this_sig = this_sig.fillna(this_sig.mean())
#    new_sig_df[sig_name] = this_sig
#sig_df = new_sig_df

In [ ]:
pnl_df = pd.DataFrame()
for i in range( model_sig_df.shape[1]):
    print(i)
    pnl_sr = pm.run_signal(sig_sr=model_sig_df.iloc[:,i],neu_sig=True)
    pnl_df[dim_vec[i]] = pnl_sr

pnl_df[0] = np.zeros(pnl_df.shape[0])

In [ ]:
window = 12
#pnl_df_avg = ( pnl_df.rolling(window=window).mean()*sqrt(12) ) /( pnl_df.rolling(window=window).std() )
pnl_df_avg = ( pnl_df.rolling(window=window).mean()*sqrt(12) )    
max_ind = pnl_df_avg.idxmax(axis=1)
min_ind = pnl_df_avg.idxmin(axis=1)
max_ind_smt = max_ind.ewm(halflife=window).mean()

rcParams['figure.figsize'] = 20,10
max_ind_smt.plot()

In [ ]:
window = 6
#pnl_df_avg = ( pnl_df.rolling(window=window).mean()*sqrt(12) ) /( pnl_df.rolling(window=window).std() )
pnl_df_avg = ( pnl_df.rolling(window=window).mean()*sqrt(12) )    
max_ind = pnl_df_avg.idxmax(axis=1)
min_ind = pnl_df_avg.idxmin(axis=1)
max_ind_smt = max_ind.ewm(halflife=window).mean()

rcParams['figure.figsize'] = 20,10
max_ind_smt.plot()

In [ ]:
(pnl_df_avg.iloc[:,-2] - pnl_df_avg.iloc[:,0] ).plot()

In [ ]:
pnl_df.mean(axis=0)*sqrt(12)/pnl_df.std(axis=0)

In [ ]:
dim_sr = pd.Series( dim_vec, index = pnl_df_avg.columns[:-1] )

In [ ]:
c = ( pnl_df.iloc[:,:-1] ).corrwith(dim_sr, axis=1) 

In [ ]:
c.ewm(halflife=24).mean().plot()

In [ ]:
import statsmodels.api as sm
lm_weights = []
lm_tstats = []
lm_pvalues = []

window = 24
pnl_df_avg = pnl_df.ewm(halflife=window).mean()
pnl_df_avg.dropna(inplace=True, axis=1)

for i in range(window,pnl_df_avg.shape[0]):
    lm = LinearRegression(fit_intercept=True)
    X = pd.DataFrame()
    X['x'] = dim_sr
    y = pnl_df_avg.iloc[i,:-1]
    #y = ( y - y.mean())/y.std()
    #X = sm.add_constant(X, prepend=False)
    model = sm.OLS(y,X)
    results = model.fit()
    lm_weights.append( results.params[0] )

    lm_tstats.append( results.tvalues[0] )
    lm_pvalues.append(results.f_pvalue)

    
dates = pnl_df.index[(window):]
print(dates.shape)
print(pnl_df_avg.shape)

lm_weights = pd.Series(lm_weights, index = dates)
lm_tstats = pd.Series(lm_tstats, index = dates)
lm_pvalues = pd.Series(lm_pvalues, index = dates)

In [ ]:
lm_weights.ewm(halflife=3).mean().plot()

In [ ]:
lm_pvalues.ewm(halflife=3).mean().plot()

In [ ]:
sr.iloc[:-1].plot()

In [ ]:
plot( dim_vec, pnl_df_avg.iloc[-1,].values[:-1], )

In [ ]:
pnl_df_avg